<center><div style="direction:rtl;font-family:B Lotus, B Nazanin, Tahoma">به نام خدا</div></center>
<h1><center><div style="direction:rtl;font-family:B Lotus, B Nazanin, Tahoma">tf.data بازنویسی نوت بوک 7 با </div></center></h1>
                         rasoul asadiyan  

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os, shutil
import pathlib

In [39]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir =  "/root/.keras/datasets/cats_and_dogs_filtered/train"
validation_dir = "/root/.keras/datasets/cats_and_dogs_filtered/validation"

In [13]:
PATH

'/root/.keras/datasets/cats_and_dogs_filtered'

In [3]:
full_data=tf.data.Dataset.list_files(str('/root/.keras/datasets/cats_and_dogs_filtered/*/*/*'))

In [4]:
for i in full_data.take(5):
  print(i)

tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.342.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/validation/cats/cat.2199.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.784.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.569.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.948.jpg', shape=(), dtype=string)


In [5]:
full_train=tf.data.Dataset.list_files(str('/root/.keras/datasets/cats_and_dogs_filtered/train/*/*'))

In [6]:
for i in full_train.take(5):
  print(i)

tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/cats/cat.931.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.203.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.389.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/cats/cat.82.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.401.jpg', shape=(), dtype=string)


In [7]:
full_validation=tf.data.Dataset.list_files(str('/root/.keras/datasets/cats_and_dogs_filtered/validation/*/*'))

In [8]:
dataset_size=len(list(full_data))# train=2000 / validation=1000
train_size=len(list(full_train))
validation_size=len(list(full_validation))
print("full_dataset_size = {}".format(dataset_size))
print("full_train_size = {}".format(train_size))
print("full_validation_size = {}".format(validation_size))

full_dataset_size = 3000
full_train_size = 2000
full_validation_size = 1000


In [9]:
class_names=np.array(['cats','dogs'])

In [10]:
class_names

array(['cats', 'dogs'], dtype='<U4')

In [14]:
def augment_data(img):
  img=tf.image.resize_with_crop_or_pad(img,180,180)
  img=tf.image.random_crop(img,size=[150,150,3])
  img=tf.image.random_brightness(img,max_delta=0.5)
  img=tf.image.random_flip_left_right(img)
  img=tf.image.random_flip_up_down(img)
  
  return img

In [15]:
def get_label(file_path):
  parts=tf.strings.split(file_path,os.path.sep)
  if parts[-2]=='cats':
    return 0
  else:
    return 1  

In [16]:
get_label('root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.412.jpg')

1

In [17]:
def load_img(imag_path):
  img=tf.io.read_file(imag_path)
  img=tf.image.decode_image(img,3,expand_animations=False)
  img=tf.cast(img,tf.float32)
  
  return img

In [18]:
def normalize(img):
  img=(img/127.5)-1
  return img

In [19]:
def resize_img(imag,height,width):
  img=tf.image.resize(imag,(height,width),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  return img

In [20]:
def load_img_label(img_path):
  imag=load_img(img_path)
  label=get_label(img_path)
  return imag,label

In [21]:
def load_imag_train(img_path):
  imag,label=load_img_label(img_path)
  imag=augment_data(imag)
  imag=normalize(imag)

  return imag,label

In [22]:
def load_img_validation(img_path):
  imag,label=load_img_label(img_path)
  imag=resize_img(imag,150,150)
  imag=normalize(imag)

  return imag,label

In [23]:
batch_size=32
shuffle_buffer_size=500

In [24]:
AUTOTUNE=tf.data.experimental.AUTOTUNE

In [25]:
train_dataset=full_train.map(load_imag_train,num_parallel_calls=AUTOTUNE)
train_dataset=train_dataset.shuffle(shuffle_buffer_size)
train_dataset=train_dataset.batch(batch_size)
train_dataset=train_dataset.prefetch(buffer_size=AUTOTUNE)

In [26]:
validation_dataset=full_validation.map(load_img_validation,num_parallel_calls=AUTOTUNE)
validation_dataset=validation_dataset.batch(batch_size)
train_dataset=train_dataset.prefetch(buffer_size=AUTOTUNE)

In [27]:
base_model=tf.keras.applications.VGG16(weights='imagenet',include_top=False,input_shape=(150,150,3))
base_model.trainable=False

58900480/58889256 [==============================] - 0s 0us/step


In [37]:
flatten_layer = tf.keras.layers.GlobalAveragePooling2D()

dense_layer = tf.keras.layers.Dense(100, activation='relu')
dropout_layer = tf.keras.layers.Dropout(0.5)
flatten = tf.keras.layers.Flatten()
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')

In [38]:
model = tf.keras.Sequential([
  base_model,
  flatten_layer,dense_layer, dropout_layer,
  flatten,
  prediction_layer
])

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 100)               51300     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 flatten (Flatten)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                        

In [43]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [41]:
for data_batch,label_batch in validation_dataset:
  print(data_batch.shape)
  print(label_batch.shape)
  
  break

(32, 150, 150, 3)
(32,)


In [44]:
h = model.fit(train_dataset,
                    epochs=10,
                    validation_data=validation_dataset)

Epoch 1/10
63/63 [==============================] - 661s 10s/step - loss: 0.6979 - accuracy: 0.6850 - val_loss: 0.3378 - val_accuracy: 0.8520
Epoch 2/10
24/63 [==========>...................] - ETA: 4:30 - loss: 0.4984 - accuracy: 0.7695

KeyboardInterrupt: ignored